NameError: name 'stopwords' is not defined

In [2]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import string

# import some modules
from numpy import linalg as LA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.corpus import words
from __future__ import division



In [ ]:
# get stop words
STOP_WORDS = stopwords.words('english')

# Helper functions
# method to remove apostrophe to make it easier to detect stop words
def remove_apostrophe(inp):
    inp = inp.replace("'", " ")
    return inp

# method to remove stop words or non-words or both
def clean_text(inp):
    new_inp=inp
    clean_inp=[]
    for char in new_inp.split():
        if char.lower() not in STOP_WORDS:
            clean_inp.append(char) #cleaning stop words only
    return ' '.join(clean_inp)

# method to clean text
def clean_matrix(df):
    #applying text clean on each sms in the matrix
    df['sms'] = df['sms'].apply(remove_apostrophe)
    df['sms'] = df['sms'].apply(clean_text)
    return df

# method to transform cleaned text to normalized feature vector
def transform_train_and_test_matrix(train_dataset_path, test_dataset_path):
    
    # load the dataframe
    df_train = pd.read_csv(train_dataset_path,encoding='latin1')
    df_test = pd.read_csv(test_dataset_path,encoding='latin1')
    
    # clean the data
    clean_dataframe_train = clean_matrix(df_train)
    clean_dataframe_test = clean_matrix(df_test)
    
    # transform labels into a binary vector
    labels_train = [1 if(df_train['label'].iloc[i]) == 'ham' else 0 for i in range(len(df_train))]
    labels_test = [1 if(df_test['label'].iloc[i]) == 'ham' else 0 for i in range(len(df_test))]
    
    #calling CountVectorizer
    transformer = CountVectorizer(decode_error='ignore')
    messages_transform_train=transformer.fit_transform(clean_dataframe_train['sms'])
    messages_transform_test=transformer.transform(clean_dataframe_test['sms'])
    #print(transformer.vocabulary_)
    #print((messages_transform.toarray()))
    
    #calling tf_idf
    tf_idf_trans = TfidfTransformer(norm='l2', smooth_idf=False)
    messages_tfidf_train = tf_idf_trans.fit_transform(messages_transform_train)
    messages_tfidf_test = tf_idf_trans.transform(messages_transform_test)
    #print((messages_tfidf.toarray()))
    
    return np.array(messages_tfidf_train.toarray()), np.array(np.reshape(labels_train,(len(labels_train),1))), np.array(messages_tfidf_test.toarray()), np.array(np.reshape(labels_test,(len(labels_test),1))